In [9]:
%pip install pyreadr
%pip install pandas
%pip install glob
%pip install folium
%pip install matplotlib
%pip install seaborn
%pip install tabulate
%pip install plotly.express
%pip install --upgrade nbformat
%pip install geopy




Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [10]:
import folium as fm
import pandas as pd
import pyreadr 
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import sys
from tabulate import tabulate
import plotly.express as px
from geopy.distance import geodesic

In [11]:
class day():
    def __init__(self, address):
        self.address = address

    def read_rds_file_per_day(self):
        rds_files = glob.glob(self.address + "/*.rds")
        if not rds_files:
            print("No .rds files found in the directory.")
            return None
        dataframes = []
        for file in rds_files:
            result = pyreadr.read_r(file)  # Read the .rds file
            df = list(result.values())[0]  # Extract the DataFrame from the result
            dataframes.append(df)

        return dataframes
    
    def print_df(self, x):
        print(tabulate(x, headers='keys', tablefmt='psql'))

    def clean_df(self, data):
    #list to remove and drop na rows
        list_to_remove = ['pedelec_battery', 'terminal_type', 'place_type', 'boardcomputer',
                      'bike_type', 'lock_types', 'active', 'state', 'electric_lock', 'bike_racks', 
                      'free_racks', 'rack_locks', 'no_registration', 'bike', 'uid', 'city']
        for i in range(len(data)):
            data[i] = data[i].drop(list_to_remove, axis=1)
            data[i] = data[i].dropna()
        return(data)
    

    def maintenance_map(self, data, folder_path, file_index):
        # Ensure the output folder exists
        os.makedirs(folder_path, exist_ok=True)

        # Get the first DataFrame in the list
        df = data[file_index]

        # Group the DataFrame by 'maintenance' and filter the group where 'maintenance' is True
        data_grouped = df.groupby(df['maintenance'])
        df_new = data_grouped.get_group(True)

        # Calculate the value counts of the 'name' column
        col_maintenance = df_new['name'].value_counts().reset_index()
        col_maintenance.columns = ['name', 'count']

        # Aggregate lat and lng by name to ensure unique index for mapping
        df_unique = df_new.groupby('name').agg({'lat': 'first', 'lng': 'first', 'time_stamp':'first'}).reset_index()

        # Merge col_maintenance with df_unique on the 'name' column
        col_maintenance = pd.merge(col_maintenance, df_unique, on='name', how='left')
        col_maintenance = col_maintenance.dropna()
    
        # Initialize a Folium map centered around the average coordinates
        map_center = [col_maintenance['lat'].mean(), col_maintenance['lng'].mean()]
        mymap = fm.Map(location=map_center, zoom_start=14)

        # Add markers to the map
        for idx, row in col_maintenance.iterrows():
            fm.Marker(
             location=[row['lat'], row['lng']],
             popup=f"{row['name']} ({row['count']} times)",
             icon=fm.Icon(icon='info-sign')
            ).add_to(mymap)

        # Save the map as an HTML file
        map_filename = os.path.join(folder_path, f'map_{file_index+1}.html')
        mymap.save(map_filename)

        return col_maintenance
    def bike_available_map(self, data, folder_path, file_index):
        # Ensure the output folder exists
        os.makedirs(folder_path, exist_ok=True)

        # Get the first DataFrame in the list
        df = data[file_index]

        # Group the DataFrame by 'maintenance' and filter the group where 'maintenance' is True
        data_grouped = df.groupby(df['maintenance'])
        df_new = data_grouped.get_group(False)

        # Calculate the value counts of the 'name' column
        val_maintenance = df_new['name'].value_counts().reset_index()
        val_maintenance.columns = ['name', 'count']

        # Aggregate lat and lng by name to ensure unique index for mapping
        df_unique = df_new.groupby('name').agg({'lat': 'first', 'lng': 'first', 'time_stamp':'first'}).reset_index()

        # Merge col_maintenance with df_unique on the 'name' column
        val_maintenance = pd.merge(val_maintenance, df_unique, on='name', how='left')
        val_maintenance = val_maintenance.dropna()
        
        # Initialize a Folium map centered around the average coordinates
        map_center = [val_maintenance['lat'].mean(), val_maintenance['lng'].mean()]
        mymap = fm.Map(location=map_center, zoom_start=14)

        # Add markers to the map
        for idx, row in val_maintenance.iterrows():
            fm.Marker(
                location=[row['lat'], row['lng']],
                popup=f"{row['name']} ({row['count']} times)",
                icon=fm.Icon(icon='info-sign')
        ).add_to(mymap)

        # Save the map as an HTML file
        map_filename = os.path.join(folder_path, f'map_{file_index+1}.html')
        mymap.save(map_filename)

        return val_maintenance
    
    
    def plot_data_xtime_ycountsum(self, data, output_folder, output_name):
        name = "bike count ..............."
        # Combine all DataFrames into one
        df_combined = pd.concat(data)
        # Pivot the DataFrame to get stations as columns
        df_pivot = df_combined.pivot_table(index='time_stamp', columns='name', values='count', fill_value=0)
        # Reset index to have time_stamp as a column for plotting
        df_pivot = df_pivot.reset_index()

        # Assuming df_pivot is your DataFrame
        df_pivot.set_index('time_stamp', inplace=True)

        # Create the interactive area plot using Plotly
        fig = px.area(
            df_pivot,
            x=df_pivot.index,
            y=df_pivot.columns,
            labels={'value': 'Count', 'time_stamp': 'Timestamp'},
            title=name
        )

        # Update layout for better readability
        fig.update_layout(
            xaxis_title="Timestamp",
            yaxis_title="Count",
            hovermode="x unified",  # Show x and y values together on hover
            legend_title="Station",
            template='plotly_dark'  # Use the dark template if desired
        )

        # Ensure the output folder exists
        os.makedirs(output_folder, exist_ok=True)

        # Save the plot as an HTML file
        output_file = os.path.join(output_folder, output_name)# "maintenance_plot.html")
        fig.write_html(output_file)

        print(f"Plot saved as {output_file}")
        
        # Optionally display the plot in a notebook or a browser
        #fig.show()
    
    def calculate_route_distance(self, coords):
        total_distance = 0.0
        for i in range(len(coords) - 1):
            total_distance += geodesic(coords[i], coords[i + 1]).kilometers
        return total_distance
    
    def generate_bike_maps(self, data, output_directory):

        os.makedirs(output_directory, exist_ok=True)
        # Concatenate and filter data
        df_concatenated = pd.concat(data, axis=0, ignore_index=True)
        df_concatenated = df_concatenated.groupby(df_concatenated['maintenance'])
        df_concatenated = df_concatenated.get_group(False)

        # Create location_info column
        df_concatenated['location_info'] = df_concatenated.apply(
            lambda row: f"{row['lat']}, {row['lng']}, {row['name']}", axis=1
        )

        # Pivot the dataframe
        df_1_pivot = df_concatenated.pivot(index='bike_number', columns='time_stamp', values='location_info')
        df_1_pivot = df_1_pivot.fillna('-')
        
        #df_1_pivot.to_csv('orgin_dest_flow.csv')
        #print_df(df_1_pivot)
        # Generate individual maps for each bike
        for bike_number in df_1_pivot.index:
            m = fm.Map(location=[df_concatenated['lat'].mean(), df_concatenated['lng'].mean()], zoom_start=13)
            coords = []
            route_info = []
            for col in df_1_pivot.columns:
                value = df_1_pivot.at[bike_number, col]
                if value != '-':
                    lat, lng, name = value.split(', ')
                    lat = float(lat)
                    lng = float(lng)
                    coords.append((lat, lng))
                    route_info.append(f"{col}: {name}")
            if coords:
                start_coords = coords[0]
                end_coords = coords[-1]

                distance = self.calculate_route_distance(coords)

                fm.Marker(
                    location=start_coords,
                    icon=fm.Icon(color='green', icon='star'),
                    popup=f"Start: {route_info[0]}"
                ).add_to(m)
                fm.Marker(
                    location=end_coords,
                    icon=fm.Icon(color='red', icon='star'),
                    popup=f"End: {route_info[-1]}<br>Distance: {distance:.2f} km"
                ).add_to(m)

                
                popup_info = f"Bike: {bike_number}<br>Route Info: {', '.join(route_info)}"
                fm.PolyLine(
                    locations=coords,
                    color='blue',
                    weight=2.5,
                    opacity=1,
                    popup=popup_info
                ).add_to(m)

                fm.PolyLine(
                    locations=coords,
                    color='red',
                    weight=2.5,
                    opacity=1,
                    popup=popup_info
                ).add_to(m)

        # Save the map to the specified directory
            map_path = os.path.join(output_directory, f'bike_{bike_number}_route.html')
            m.save(map_path)

        print(f'Individual bike maps have been saved to {output_directory}.')

    
    def time_based_flow(self, data, output_directory):
        os.makedirs(output_directory, exist_ok=True)
        
        # Concatenate and filter data
        df_concatenated = pd.concat(data, axis=0, ignore_index=True)
        df_concatenated = df_concatenated[df_concatenated['maintenance'] == False]

        # Create location_info column
        df_concatenated['location_info'] = df_concatenated.apply(
            lambda row: f"{row['lat']}, {row['lng']}, {row['name']}", axis=1
        )

        # Pivot the dataframe
        df_1_pivot = df_concatenated.pivot(index='bike_number', columns='time_stamp', values='location_info')
        df_1_pivot = df_1_pivot.fillna('-')

        # Create map centered around the mean latitude and longitude
        m = fm.Map(location=[df_concatenated['lat'].mean(), df_concatenated['lng'].mean()], zoom_start=13)

        # Colors to differentiate routes
        colors = ['blue', 'red', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']
        
        # Loop through each bike
        for i, bike_number in enumerate(df_1_pivot.index):
            coords = []
            route_info = []
            previous_location = None
            
            for col in df_1_pivot.columns:
                value = df_1_pivot.at[bike_number, col]
                if value != '-':
                    lat, lng, name = value.split(', ')
                    lat = float(lat)
                    lng = float(lng)
                    current_location = (lat, lng)
                    
                    # Only add if the location has changed
                    if current_location != previous_location:
                        coords.append(current_location)
                        route_info.append(f"{col}: {name}")
                        previous_location = current_location
            
            # Skip if no movement (only one unique location)
            if len(coords) > 1:
                # Calculate the route distance
                distance = self.calculate_route_distance(coords)

                # Define a color for the PolyLine
                route_color = colors[i % len(colors)]
                
                # Add PolyLine for the route
                popup_info = f"Bike: {bike_number}<br>Route Info: {', '.join(route_info)}"
                fm.PolyLine(
                    locations=coords,
                    color=route_color,
                    weight=2.5,
                    opacity=1,
                    popup=popup_info
                ).add_to(m)

                # Add markers for start and end points
                start_coords = coords[0]
                end_coords = coords[-1]

                # Start marker
                fm.Marker(
                    location=start_coords,
                    icon=fm.Icon(color='green', icon='play'),
                    popup=f"Start: {route_info[0]}"
                ).add_to(m)

                # End marker
                fm.Marker(
                    location=end_coords,
                    icon=fm.Icon(color='red', icon='stop'),
                    popup=f"End: {route_info[-1]}<br>Distance: {distance:.2f} km"
                ).add_to(m)

        # Save the map
        map_path = os.path.join(output_directory, 'bike_routes_per_time_map.html')
        m.save(map_path)
        return map_path
    
    


    
    


    

In [13]:
if __name__ == "__main__":
    print(os.getcwd())
    path = "D:\\df\\2023-06-03"
    
    june_03 = day(path)
    data_june_03 = june_03.read_rds_file_per_day()
    cleaned_data_june03 = june_03.clean_df(data_june_03)
    output_folder_maps = 'D:\\df\\maps_maintenance_june03'
    
    column_maintenance_data = []
    # Loop through each DataFrame and create/save a map
    for i in range(len(cleaned_data_june03)):
       column_maintenance_data.append(june_03.maintenance_map(cleaned_data_june03, output_folder_maps, i))
    
    output_folder_plot = 'D:\\df\\plots'
    june_03.plot_data_xtime_ycountsum(column_maintenance_data, output_folder_plot, "maintenance_plot_2023-06-03.html")
    
    output_folder_maps_bikeavailable = 'D:\\df\\maps_bike_available'
    bike_available_data = []
    # Loop through each DataFrame and create/save a map
    for i in range(len(cleaned_data_june03)):
      bike_available_data.append(june_03.bike_available_map(cleaned_data_june03, output_folder_maps_bikeavailable, i))
    june_03.plot_data_xtime_ycountsum(bike_available_data, output_folder_plot, "available_bike_plot_june-03.html")

    june_03.generate_bike_maps(cleaned_data_june03, 'D:\\df\\maps_bike_movement\\june_03')

    time_1 = 480 #(8*60)#this should be checked before
    time_2 = 540 #(9*60)#this should be checked before 
    #print(cleaned_data_june03[time_1:time_2])#to check whether the time frame is from 08:00 to 08:59
    june_03.time_based_flow(cleaned_data_june03[time_1:time_2],"D:\\df\\time_based\\june_03")

   
    

    

D:\Student-job\my_virtual_env


C:\Users\Fenin Jose\AppData\Local\Temp\ipykernel_9576\1520089118.py:48: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\Fenin Jose\AppData\Local\Temp\ipykernel_9576\1520089118.py:48: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\Fenin Jose\AppData\Local\Temp\ipykernel_9576\1520089118.py:48: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\Fenin Jose\AppData\Local\Temp\ipykernel_9576\1520089118.py:48

Plot saved as D:\df\plots\maintenance_plot_2023-06-03.html


C:\Users\Fenin Jose\AppData\Local\Temp\ipykernel_9576\1520089118.py:87: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\Fenin Jose\AppData\Local\Temp\ipykernel_9576\1520089118.py:87: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\Fenin Jose\AppData\Local\Temp\ipykernel_9576\1520089118.py:87: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\Fenin Jose\AppData\Local\Temp\ipykernel_9576\1520089118.py:87

Plot saved as D:\df\plots\available_bike_plot_june-03.html
Individual bike maps have been saved to D:\df\maps_bike_movement\june_03.
